In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import * 
from ngsolve.meshes import MakeStructured2DMesh
from ngsolve.solvers import GMRes
from netgen.geom2d import SplineGeometry
import matplotlib.pyplot as plt
import scipy.sparse as sp
import matplotlib.pylab as plt

In [2]:
# functions for differential operators on manufactured solutions 

coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """

    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """

    if cf.dim == 1: # if the functions is getting handed a scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of a vector coefficient function """

    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

def createGeometry(h):
    geo = SplineGeometry()
    geo.AddRectangle((0,0),(1,1),bc="rectangle")
    ngmesh = geo.GenerateMesh(maxh=h)
    return Mesh(ngmesh)

In [3]:
def standardHodgeLaplace(mesh, g, order):

    H_curl = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl) space
    H_1 = H1(mesh, order=order)     # For 0-forms, H1 space
    fes = H_curl * H_1
    (u, p), (v, q) = fes.TnT()

    n = specialcf.normal(mesh.dim)
    curlg = GCurl(g)
    gamma_n_g = CF((curlg*n[1], -curlg*n[0]))
    
    f = CF(GCurl(GCurl(g)) - GGrad(GDiv(g)) )

    B, F = BilinearForm(fes), LinearForm(fes) 
    

    B += curl(u)*curl(v)*dx
    B += grad(p)*v*dx 
    B += grad(q)*u*dx
    B += - p * q * dx

    F += f*v*dx
    F += - gamma_n_g * v.Trace() * ds
    F += (g * n) * q * ds

    B.Assemble(); F.Assemble()

    #with TaskManager(): 
    B.Assemble()
    F.Assemble()
    sol = GridFunction(fes)
    
    res = F.vec-B.mat * sol.vec
    inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
    sol.vec.data += inv * res

    # rows, cols, vals = B.mat.COO()
    # B_mat = sp.csr_matrix((vals, (rows, cols)))
    # plt.spy(B_mat)

    #plt.plot(F.vec)

    return sol
    


In [4]:
n = 0.01
order = 2
g = CF((sin(pi*x)*cos(pi*y)*x*(1-x)*y*(1-y), sin(pi*y)*cos(pi*x)*x*(1-x)*y*(1-y)))

mesh = createGeometry(n)
solution = standardHodgeLaplace(mesh, g, order)

In [5]:
Draw(solution.components[0], mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene